# Imports

First we import the discrete GRN dependencies.

In [ ]:
from discrete_motif import DiscreteGrnMotif
import discrete_motif_functions as functions
import discrete_motif_operations as operations
import discrete_motif_measures as measures
import discrete_motif_generator as generator
import discrete_motif_plotting as visualize

# Data imports

# Experiments

## Correlation between synergy and nudge resilience in random motif

First, we

In [ ]:
# create a network motif
motifs, indegree_avg = generator.generate_motifs(50, 3)
print("average indegree is "+str(indegree_avg))

# make a plot with WMS synergy
visualize.scatterplot_synergy_nudgeimpact(motifs, 3, 0.7, False);

## More synergy in biological motif than in random